# Homework_2021_03_23
截止日期：03.29 

## 第一部分：XML解析 

### 1.1 读取并解析tang300.xml文件，首先请按行打印出来文件的内容，10行即可。
关于xml解析的相关代码，可以参看https://www.runoob.com/python/python-xml.html

In [ ]:
with open('tang300.xml','r') as f:
    for i in range(10):
        print(f.readline().strip())

### 1.2 各保存20首李白、杜甫、白居易、王維（維是繁体）的诗（题目以及内容）。要求为每首诗歌生成独立的数据对象，包括 标题、作者、诗内容（按行）的元组（tuple）

In [ ]:
# import xml.dom.minidom
# import xml.sax
import re
'''
使用minidom解析器打开 XML 文档，当然使用sax也行。或者也可以尝试自己写解析算法，比如正则表达式。
不同的方法实现得好都有机会被作为作业样本并获得加分
'''
# DOMTree = xml.dom.minidom.parse("tang300.xml")
# collection = DOMTree.documentElement
 
# # 在集合中获取所有诗句
# poems = collection.getElementsByTagName("作业用唐诗")
with open('tang300.xml','r') as f:
    poem_xml=f.read()

poet_re=lambda name:re.compile('<作业用唐诗>\n<ID>\d+</ID>\n<Poem_id>\d+</Poem_id>\n<line_number>-100</line_number>\n<contance>##(.+)</contance>\n</作业用唐诗>\n<作业用唐诗>\n<ID>\d+</ID>\n<Poem_id>\d+</Poem_id>\n<line_number>-1</line_number>\n<contance>\$\$(%s)</contance>\n</作业用唐诗>\n<作业用唐诗>\n<ID>\d+</ID>\n<Poem_id>\d+</Poem_id>\n<line_number>1</line_number>\n<contance>(.+)</contance>\n</作业用唐诗>\n<作业用唐诗>\n<ID>\d+</ID>\n<Poem_id>\d+</Poem_id>\n<line_number>2</line_number>\n<contance>(.+)</contance>\n</作业用唐诗>\n<作业用唐诗>\n<ID>\d+</ID>\n<Poem_id>\d+</Poem_id>\n<line_number>3</line_number>\n<contance>(.+)</contance>\n</作业用唐诗>\n<作业用唐诗>\n<ID>\d+</ID>\n<Poem_id>\d+</Poem_id>\n<line_number>4</line_number>\n<contance>(.+)</contance>\n</作业用唐诗>\n'%name)

poems=dict()
for i in '李白、杜甫、白居易、王維'.split('、'):
    poems[i]=poet_re(i).findall(poem_xml)[:20]
## 请在这里打印出保存的任意一位诗人的诗的数据对象
poems['李白']

### 1.3 实现search_poem函数，该函数打印你保存的指定作者的诗歌的标题，并返回一个list。

In [ ]:
def search_poem(author):
    '''
    可以通过循环或列表生成式实现
    '''
    title_list=[poems[author][i][0] for i in range(20)]
    return title_list
search_poem("李白")

## 第二部分 发布者-订阅者模式

实现 设计模式中的观察者模式，本部分将以第一部分中提取的诗歌信息作为主题。

观察者模式也被称为发布-订阅（Publish/Subscribe）模式。当这个主题对象状态变化时，会通知所有观察者对象并作出相应处理逻辑。

在本问题中，可以形象地理解成，Publisher为诗歌发布中心，Reader为订阅相应诗人的诗歌的人。需要实现订阅，退订，Publisher可以发布新增、删除诗歌的消息以及当前各位诗人的诗歌的总数。当Publisher发布消息时，它需要把需要推送给所有订阅的Reader。


**Publisher（发布者）**

    - 保存订阅的读者
    - 记录四位诗人各自诗歌的总数
    - 发布四位诗人新诗的消息，每次只需要发布一首
    - 发布新消息时，发布给所有已订阅的读者   
   只有一位发布者Publisher
   
**Reader（订阅者）**

    - 订阅/取消订阅 发布者（注意同一个发布者不要重复订阅）
    - 保存发布者发来的诗歌
关于保存消息，读者分为**两种类型**：
- Type1：保存所有消息，这意味着这类读者需要将四位诗人的诗歌分类。
- Type2：只保存最新的消息（也就是说只需要保存一首诗）。 

要求实例化三个Reader: Alice, Bob, Carol，其中Alice为Type1，Bob&Carol为Type2.

要求Reader实现printStatistics方法来打印保存的诗歌。

要求实现Publisher的__str__方法，输出自己现有的Reader和发出的诗歌数目。

下面给出的代码作为参考，可以不按照下面的参考实现。

#### 参考资料

[1. Observer pattern Wiki](https://en.wikipedia.org/wiki/Observer_pattern)   
[2. 设计模式之观察者模式（c++）](https://www.cnblogs.com/carsonzhu/p/5770253.html)

### 2.1 完成上述Publisher & Reader的代码，先不需要考虑多进程

In [ ]:
class BasePublisher(object):
    def __init__(self,name):
        self.name=name
        self.reader=[]
        self.published=0
        self.poems={'李白':0,'杜甫':0,'白居易':0,'王維':0}

    def subscribeReader(self, reader):
        self.reader.append(reader)

    def unsubscribeReader(self, reader):
        self.reader.remove(reader)

    def notifyReader(self,author, poem):
        raise NotImplementedError
            
    def __str__(self):
        return '现有的Reader: '+','.join([i.name for i in self.reader])+'\n发出的诗歌数目'+str(self.poems)


class Publisher(BasePublisher):
    def notifyReader(self,author, poem):
        self.poems[author]+=1
        for i in self.reader:
            i.receivePoem(self.name,poem,author)

In [ ]:
import threading
class BaseReader(object):

    def __init__(self):
        raise NotImplementedError
        
    def subscribeToPublisher(self, publisher:Publisher):
        publisher.subscribeReader(self)
 
    def unsubscribeToPublisher(self, publisher:Publisher):
        publisher.unsubscribeReader(self)
 
    def __str__(self):
        raise NotImplementedError
        
    def receivePoem(self, publisher, poem , author):
        raise NotImplementedError

    def printStatistics(self):
        raise NotImplementedError

In [ ]:
# 第一种读者
class ReaderType1(BaseReader):

    def __init__(self, _name):
        self.poems={'李白':[],'杜甫':[],'白居易':[],'王維':[]}
        self.name=_name

    def receivePoem(self, publisher, poem, author):
        print('thread id :{}'.format(threading.current_thread().name)) 
        self.poems[author].append(poem)
        
    def printStatistics(self):
        print(self.poems)

    def __str__(self):
        return self.name+'接收的诗'+str(self.poems)
        

# 第二种读者
class ReaderType2(BaseReader):

    def __init__(self, _name):
        self.poem=None
        self.name=_name

    def receivePoem(self, publisher, poem , author):
        print('thread id :{}'.format(threading.current_thread().name)) 
        self.poem=(author,poem)
        

    def printStatistics(self):
        print(self.poem)

    def __str__(self):
        return self.name+'接收的诗'+str(self.poem)

In [ ]:
Publisher = Publisher("Publisher")
Alice=ReaderType1('Alice')
Bob=ReaderType2('Bob')
Carol=ReaderType2('Carol')

In [ ]:
Alice.subscribeToPublisher(Publisher)
Bob.subscribeToPublisher(Publisher)
## 请在这里利用Publisher任意发出几首诗来测试你的代码
Publisher.notifyReader('李白','贈汪倫  白遊涇縣桃花潭. 村人汪倫常醞美酒以待白')
Publisher.notifyReader('白居易',search_poem('白居易')[2])
Publisher.notifyReader('杜甫',search_poem('杜甫')[3])

In [ ]:
Alice.printStatistics()
Bob.printStatistics()
Carol.printStatistics()
print(Publisher)

In [ ]:
Carol.subscribeToPublisher(Publisher)
Bob.unsubscribeToPublisher(Publisher)
##再任意发出几首诗进行测试
Publisher.notifyReader('白居易',search_poem('白居易')[4])
Publisher.notifyReader('杜甫',search_poem('杜甫')[0])
Publisher.notifyReader('白居易',search_poem('白居易')[0])
Publisher.notifyReader('杜甫',search_poem('杜甫')[8])

In [ ]:
Alice.printStatistics()
Bob.printStatistics()
Carol.printStatistics()
print(Publisher)

### 2.2 修改Publisher部分代码，用多线程（或者多进程）的方式发送消息。本部分为选做部分。

In [ ]:
class MultiThreadPublisher(BasePublisher):
    
    def notifyReader(self,author, poem):
        self.poems[author]+=1
        def single_notify(receiver):
            receiver.receivePoem(self.name,poem,author)
        thread_list=[]
        for i in self.reader:
            thread_list.append(threading.Thread(target=single_notify,args=(i,)))
        for i in thread_list:
            i.start()
        for i in thread_list:
            i.join()

In [ ]:
MultiThreadPublisher = MultiThreadPublisher("MultiPublisher")
Alice.subscribeToPublisher(MultiThreadPublisher)
Bob.subscribeToPublisher(MultiThreadPublisher)
Carol.subscribeToPublisher(MultiThreadPublisher)
print(Alice)

In [ ]:
##请发出几首诗测试
MultiThreadPublisher.notifyReader('王維',search_poem('王維')[2])
MultiThreadPublisher.notifyReader('王維',search_poem('王維')[3])
MultiThreadPublisher.notifyReader('白居易',search_poem('白居易')[4])

In [ ]:
Alice.printStatistics()
Bob.printStatistics()
Carol.printStatistics()